# Run Analysis

Data sources are stored as arbitrarily deep directories in /data. A data source is a directory with an index file which contains front matter such as filetype, cadence, etc related to how to access the data and when/whether to update the data automatically.  

Some data sources have a static cadence, meaning they wont be automatically updated by this notebook.  

Others specify an update cadence, a time last updated, and a method of updating which may include things like an api key, etc.  

An analysis, likewise will be an arbitrarily deep sirectory wihtin /analysis which contains an index file with front matter like title and dependencies.  

Dependencies are paths to data sources or other analyses. Whenever an analysis was last modified before any one of its dependencies, the analysis is stale and needs to be run again.  

In [7]:

#### Make sure all required packages are installed and imported

import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None, required: bool = True):
    try:
        return importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        try:
            subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
        except Exception:
            if required:
                raise
    mod = importlib.import_module(import_name or pkg_name)
    globals()[import_name or pkg_name] = mod
    return mod

_ensure('pandas')
_ensure('requests')
_ensure('feedparser')
_ensure('textblob')
_ensure('pyyaml', 'yaml')
_ensure('jupyter', required=False)
_ensure('nbconvert', required=False)
print('All dependencies ready.\n')


All dependencies ready.



## Update Data Sources

This cell needs to find all of the /data index files, check whether that data source needs to be updated, and then do whatever updates are appropriate.  



In [8]:
from pathlib import Path
import datetime as dt
import os, re, shutil, json, feedparser, textblob
import pandas as pd, requests, urllib.parse, yaml
from typing import Optional

CADENCE_SECONDS = {
    'hourly': 3600,
    'daily': 86400,
    'weekly': 604800,
    'monthly': 2592000,
    'quarterly': 7776000,
}

def substitute_date_tokens(url: str) -> str:
    def _replace(m):
        return dt.date.today().strftime(m.group(1).strip())
    return re.sub(r"\[date\s+([^\]]+)\]", _replace, url)

def add_apikey(url: str, env_var: Optional[str]) -> str:
    if env_var:
        key = os.getenv(env_var)
        if key:
            sep = '&' if '?' in url else '?'
            return f"{url}{sep}api_key={urllib.parse.quote_plus(key)}"
    return url

def _parse_meta(path: Path):
    text = path.read_text()
    m = re.search(r'^---\n(.*?)\n---\n?', text, re.S)
    if m:
        try:
            meta = yaml.safe_load(m.group(1)) or {}
        except Exception as e:
            print(f'Error parsing metadata in {path}:', e)
            raise
        body = text[m.end():]
    else:
        meta, body = {}, text
    return meta, body

def _write_meta(path: Path, meta: dict, body: str):
    path.write_text('---\n' + yaml.safe_dump(meta, sort_keys=False).strip() + '\n---\n' + body)

def updateData(path: str):
    base = Path(path)
    now = dt.datetime.now()
    today = now.date()
    updated = []
    for idx_file in base.rglob('index.md'):
        meta, body = _parse_meta(idx_file)
        url = str(meta.get('url', '')).strip()
        if not url or url.lower() in ('n/a', 'na', 'none'):
            continue
        filetype = str(meta.get('filetype', '')).strip().lstrip('.')
        cadence = str(meta.get('cadence', 'monthly')).lower()
        api_key = meta.get('api_key')
        last_fetch = pd.to_datetime(meta.get('last_fetched')) if meta.get('last_fetched') else None
        min_age = CADENCE_SECONDS.get(cadence, 2592000)
        folder = idx_file.parent
        output_ext = 'json' if filetype in ('rss', 'xml') else filetype
        latest_fp = folder / f'latest.{output_ext}'
        dated_fp = folder / (f"{now:%Y-%m-%d-%H}.{output_ext}" if cadence == 'hourly' else f"{today:%Y-%m-%d}.{output_ext}")
        if latest_fp.exists() and last_fetch and (now - last_fetch).total_seconds() < min_age:
            if dated_fp.exists() and latest_fp.read_bytes() != dated_fp.read_bytes():
                shutil.copyfile(dated_fp, latest_fp)
            continue
        req_url = add_apikey(substitute_date_tokens(url), api_key)
        try:
            r = requests.get(req_url, timeout=30, headers={'User-Agent': 'Mozilla/5.0'})
            r.raise_for_status()
            if filetype in ('rss', 'xml'):
                feed = feedparser.parse(r.content)
                entries = []
                for e in feed.entries:
                    txt = ' '.join(filter(None, [e.get('title'), e.get('summary')]))
                    pol = textblob.TextBlob(txt).sentiment.polarity
                    entries.append({'title': e.get('title'), 'link': e.get('link'), 'published': e.get('published'), 'sentiment': pol})
                content = json.dumps({'entries': entries}, ensure_ascii=False, indent=2).encode('utf-8')
            else:
                content = r.content
            dated_fp.write_bytes(content)
            shutil.copyfile(dated_fp, latest_fp)
            meta['last_fetched'] = now.isoformat(timespec='minutes')
            _write_meta(idx_file, meta, body)
            updated.append(str(folder.relative_to(base)))
        except Exception as e:
            print('Failed to fetch', folder, e)
    if updated:
        print('Updated:', ', '.join(updated))
    else:
        print('Everything up to date.')

updateData('./data')


Failed to fetch data\news\business\wapo\news-business-wapo 502 Server Error: Bad Gateway for url: https://feeds.washingtonpost.com/rss/business
Failed to fetch data\news\politics\startribune\news-us-politics-startribune 404 Client Error: Not Found for url: https://www.startribune.com/politics/index.rss2
Failed to fetch data\news\politics\wapo\news-us-politics-wapo HTTPSConnectionPool(host='www.washingtonpost.com', port=443): Read timed out. (read timeout=30)
Failed to fetch data\news\us\wapo\news-us-wapo 502 Server Error: Bad Gateway for url: https://feeds.washingtonpost.com/rss/national
Updated: news\africa\bbc\news-africa-bbc, news\africa\nyt\news-africa-nyt, news\americas\nyt\news-americas-nyt, news\asia\bbc\news-asia-bbc, news\asia\dw\news-asia-dw, news\asia\nyt\news-asia-nyt, news\business\bbc\news-business-bbc, news\business\chitri\news-business-chi-tribune, news\business\dw\news-business-dw, news\business\latimes\latimes-business, news\business\nypost\news-business-nypost, news\

## Update Analyses

This cell needs to do the same type of scan across all the analyses in /analysis. It needs to iterate across all the analyses and check the time last modified for all the dependencies. If any dependency was modified more recently than the analysis, then the analysis needs to be run again. The time last modified of the analysis is the most recent file modification time in the analysis directory, because the analysis directory will contain some arbitrary number of output files.  

Because some analyses will list other analyses as dependencies, this loop of checking across all of the analyses needs to keep running until none of them have anything to do, up to some reasonable limit of times to prevent arbitrary recursion.  

In [9]:
from pathlib import Path
import json, re, subprocess, sys, yaml
from typing import List

def _parse_meta(path: Path):
    text = path.read_text()
    m = re.search(r'^---\n(.*?)\n---\n?', text, re.S)
    if m:
        try:
            meta = yaml.safe_load(m.group(1)) or {}
        except Exception as e:
            print(f'Error parsing metadata in {path}:', e)
            raise
        body = text[m.end():]
    else:
        meta, body = {}, text
    return meta, body

def _latest_mtime(p: Path) -> float:
    if p.is_file():
        return p.stat().st_mtime
    mt=[f.stat().st_mtime for f in p.rglob('*') if f.is_file()]
    return max(mt) if mt else p.stat().st_mtime

def updateAnalyses(path: str):
    repo_dir = Path('.').resolve()
    analysis_dir = repo_dir / path
    def build_dep_map():
        dep_map={}
        for meta in analysis_dir.rglob('index.md'):
            info,_ = _parse_meta(meta)
            deps=[(repo_dir/d).resolve() for d in info.get('dependencies',[])]
            for nb in meta.parent.glob('*.ipynb'):
                dep_map[nb]=deps
        return dep_map
    def outdated(nb,deps):
        nb_m=_latest_mtime(nb)
        return any(_latest_mtime(d)>nb_m for d in deps)
    def execute(nb: Path):
        import shutil
        if not shutil.which('jupyter'):
            print('jupyter not available - skipping', nb)
            return
        cmd=[sys.executable,'-m','jupyter','nbconvert','--to','notebook','--inplace','--execute','--ExecutePreprocessor.timeout=600',str(nb)]
        subprocess.run(cmd, check=False)
    for _ in range(10):
        dep_map=build_dep_map()
        outdated_nbs=[nb for nb,deps in dep_map.items() if deps and outdated(nb,deps)]
        json.dump({'outdated_notebooks':[str(nb) for nb in outdated_nbs]}, open('dependencies.json','w'), indent=2)
        if not outdated_nbs:
            print('Everything up to date.')
            break
        for nb in outdated_nbs:
            execute(nb)
updateAnalyses('./analysis')


Everything up to date.
